Title: Dataset Creation SPEED RUN - Live Coding With Python & Pandas

Source: Rob Mulla YouTube Channel

Author (Original Tutorial): Rob Mulla

URL: https://www.youtube.com/watch?v=wiiCUsGgZx0

Date of Implementation: 2024-12-27

Description:
    Adapted implementation of dataset creation from ECB financial statement.

# Goal of This Notebook
1. Pull links to the most recent data from the SEC website:
   - https://www.bundesbank.de/de/publikationen/ezb/wirtschaftsberichte
2. Download the pdfs from the website.
3. Extract financial balance and debt data for Germany.
4. Save as CSV and Parquet format.
5. Display data in interactive plot.

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import re 
import glob
from pypdf import PdfReader
from tqdm.notebook  import tqdm
from os import listdir
from os.path import isfile, join

In [2]:
def getHTMLDocument(url):
    return requests.get(url).text

In [3]:
# Get HTML
html_document = getHTMLDocument('https://www.bundesbank.de/de/publikationen/ezb/wirtschaftsberichte')

# Create soap object 
soup = BeautifulSoup(html_document, 'html.parser') 

# Get lines related to links (indicated by 'href')
links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))

## Filter the links using a single list comprehension
#filtered_pdfs = [
#    pdf for pdf in full_pdfs if re.search(r'\bezb-wb-data\b', pdf)
#]

# Filter for wanted .pdf links
pdfs = [r for r in links if str(r).endswith('.pdf') and 'data' in r]
# Exclude links which are not 'ezb-wb-data'
pdfs = [pdf for pdf in pdfs if 'ezb-wb-data' in pdf]

## Filter links using regex
#filtered_pdfs = [pdf for pdf in full_pdfs if re.search(r'\bezb-wb-data\b', pdf)]

# Create set (remove duplicate entries etc, unpack with * to convert into list again)
pdfs = [*set(pdfs)]
full_pdfs = ['https://www.bundesbank.de' + c for c in pdfs]
print(len(full_pdfs), full_pdfs)

10 ['https://www.bundesbank.de/resource/blob/900008/1004383626794b3e8e3e054e9b6cffdb/472B63F073F071307366337C94F8C870/2024-05-ezb-wb-data.pdf', 'https://www.bundesbank.de/resource/blob/900018/9844043fc724d5efe152178c487b80b9/472B63F073F071307366337C94F8C870/2024-02-ezb-wb-data.pdf', 'https://www.bundesbank.de/resource/blob/900000/d527d1d2b61980bbe68d8215927fa9e8/472B63F073F071307366337C94F8C870/2023-08-ezb-wb-data.pdf', 'https://www.bundesbank.de/resource/blob/900006/27e14b08b34ced3121986e5c70f4bc13/472B63F073F071307366337C94F8C870/2024-06-ezb-wb-data.pdf', 'https://www.bundesbank.de/resource/blob/900012/449696afadb4033788dd43192d801d91/472B63F073F071307366337C94F8C870/2024-03-ezb-wb-data.pdf', 'https://www.bundesbank.de/resource/blob/916540/ee016565599e4a58cc48e5eb809bedef/472B63F073F071307366337C94F8C870/2023-06-ezb-wb-data.pdf', 'https://www.bundesbank.de/resource/blob/900004/7aa6c9ebde6618aecdefd050b0d73a64/472B63F073F071307366337C94F8C870/2024-07-ezb-wb-data.pdf', 'https://www.bun

In [4]:
# Load binary content from links and store as pdf (use progress bar from tqdm)
total_files = len(full_pdfs)
with tqdm(total=total_files, desc="Downloading PDFs") as pbar:
    for link in full_pdfs:
        fname = link.split('/')[-1].split('.')[0]
        print("Downloading file: ", fname)
    
        # Get response object for link
        response = requests.get(link)

        # Write content in pdf file
        with open(fname + ".pdf", 'wb') as pdf:
            pdf.write(response.content)
        pbar.update(1)

In [5]:
# Get all downloaded PDF files
pdf_files = [f for f in glob.glob("./*.pdf") if "ezb-wb-data" in f]
pdf_data = {}
for f in pdf_files:
    print("Extracting raw data from ", f)
    doc = PdfReader(f)
    # Access second last page where stats of interest are located
    page = doc.pages[-2]
    text = page.extract_text()
    text = text.replace('\xad', '')
    # Regex, extract data between the indicator lines
    pattern = r"EZB, Wirtschaftsbericht.*?\n(.*?)(?=\n6 Entwicklung der öffentlichen Finanzen)"
    # Search with line breaks
    match = re.search(pattern, text, re.S)
    if match:
        data = match.group(1)  # get lines
        data = data.strip()  # remove trailing spaces
    else:
        print("No matches!")
    pdf_data[f] = data

Extracting raw data from  ./2024-03-ezb-wb-data.pdf
Extracting raw data from  ./2023-07-ezb-wb-data.pdf
Extracting raw data from  ./2024-05-ezb-wb-data.pdf
Extracting raw data from  ./2024-07-ezb-wb-data.pdf
Extracting raw data from  ./2024-04-ezb-wb-data.pdf
Extracting raw data from  ./2023-08-ezb-wb-data.pdf
Extracting raw data from  ./2024-06-ezb-wb-data.pdf
Extracting raw data from  ./2023-06-ezb-wb-data.pdf
Extracting raw data from  ./2024-02-ezb-wb-data.pdf
Extracting raw data from  ./2024-01-ezb-wb-data.pdf


In [15]:
def fix_double_year_line(data: dict, key: str, sub_id: str, typ:str)-> None:
    raw_str = data[key][sub_id][typ]
    lines = raw_str.split("\n") # split strings by line breaks "\n"
    lines = [item for item in lines if item] # remove empty strings ''
    header = lines[0] # countries
    data = lines[1:] # remaining data
    # first data row contains two years, there is no linebreak, split them into two lines
    corrected_data = []
    left_line= [] 
    right_line = []
    line_elements = lines[1].split() # get line of interest
    # iter elements in line, find missing line break point by element length
    for el in line_elements: 
        if len(right_line) > 0:
            right_line.append(el)
        elif len(el) > 2 and "20" in el:
            idx = el.find("2019") # TODO: HARDCODED, make adaptable
            left_line.append(el[:idx])
            right_line.append(el[idx:])
        else:
            left_line.append(el)
    # get remaining data
    remaining_data = ["".join(el) for el in data[1:]]
    str_remaining_data = "\n".join(remaining_data)
    # merge parts into single string
    corrected_data.append(header)
    corrected_data.append(" ".join(left_line))
    corrected_data.append(" ".join(right_line))
    corrected_data.append("".join(str_remaining_data))
    formatted_text = "\n".join(corrected_data)
    # overwrite specific dict value
    tab_dict[key][sub_id][typ] = formatted_text
    

# Establish dict with subtables in order to merge them into one df
tab_dict = {}
for key, val in pdf_data.items():
    tab_dict[key] = {}
    split  = re.split(r'(\nLettland.*?\n)', val, maxsplit=1)
    first_half = split[0]
    second_half = split[1] + split[2]
    halfs = [first_half, second_half]
    for id, half in enumerate(halfs):
        tab_dict[key][id] = {}
        parts = re.split(r'(\nFinanzierungssaldo.*?\n|\nVerschuldung.*?\n)', half)
        balance_tab = parts[0] + parts[2]
        debt_tab = parts[0] + parts[4]
        tab_dict[key][id]["balance"] = balance_tab
        #print(tab_dict[key][id]["balance"])
        tab_dict[key][id]["debt"] = debt_tab

# iter through pdf's
for key, val in tab_dict.items():
    # iter through first and second table half
    for sub_id in [0, 1]:
        # iter through both content types
        for typ in ["balance", "debt"]:
            fix_double_year_line(tab_dict, key, sub_id, typ)
            print(tab_dict[key][sub_id][typ])
            break


Belgien Deutschland Estland Irland Griechenland Spanien Frankreich Kroatien Italien Zypern
1 2 3 4 5 6 7 8 9 10
2019 2,0 1,5 0,1 0,5 0,9 3,1 3,1 0,2 1,5 0,9
2020 8,9 4,3 5,4 5,0 9,7 10,1 9,0 7,3 9,6 5,7
2021 5,4 3,6 2,5 1,5 7,0 6,7 6,5 2,5 8,8 1,9
2022 3,5 2,5 1,0 1,7 2,4 4,7 4,8 0,1 8,0 2,4
2022 Q4 3,5 2,5 1,0 1,7 2,4 4,7 4,8 0,1 8,0 2,4
2023 Q1 3,9 3,0 1,3 2,0 2,5 4,4 4,6 0,2 8,1 3,0
Q2 4,0 3,1 1,7 2,2 2,4 4,6 4,9 0,4 7,9 3,4
Q3 4,1 2,7 2,2 1,9 1,2 4,4 4,8 0,3 6,8 3,2
Lettland Litauen Luxemburg Malta Niederlande Österreich Portugal Slowenien Slowakei Finnland
11 12 13 14 15 16 17 18 19 20
2019 0,5 0,5 2,2 0,5 1,8 0,6 0,1 0,7 1,2 0,9
2020 4,5 6,5 3,4 9,6 3,7 8,0 5,8 7,6 5,4 5,6
2021 7,2 1,1 0,6 7,5 2,2 5,8 2,9 4,6 5,2 2,8
2022 4,6 0,7 0,3 5,7 0,1 3,5 0,3 3,0 2,0 0,8
2022 Q4 4,6 0,7 0,3 5,6 0,1 3,5 0,3 3,0 2,0 0,5
2023 Q1 4,4 1,2 0,6 4,8 0,1 3,3 0,1 3,2 2,6 0,4
Q2 3,0 1,2 0,7 4,2 0,2 3,6 0,0 3,2 3,4 1,1
Q3 3,3 1,1 0,4 3,4 0,1 3,5 0,5 3,5 4,7 1,5
Belgien Deutschland Estland Irland Griec

In [19]:
def str_data_to_df(data: dict, key: str, typ: str, sub_id:str)-> pd.DataFrame:
    raw_data = data[key][sub_id][typ]
    lines = raw_data.strip().split("\n")
    countries = lines[0].split()
    numbers = lines[1].split()
    countries_with_numbers = [f"{country} ({number})" for country, number in zip(countries,numbers)]

    # Fix quarter issue
    data_rows = []
    last_year = None # Track last mentioned year

    for line in lines[2:]:
        row = line.split()
        print(len(row))
        if len(row) == len(countries_with_numbers) + 2: 
            # Year and Quarter
            time = f"{row[0]} {row[1]}" # Combine
            last_year = row[0] # Update year
            print(last_year)
            data_rows.append([time] + row[2:])
        elif len(row) == len(countries_with_numbers) + 1: 
            if 'Q' in row[0]:
                time = f"{last_year} row[0]"
                data_rows.append([time] + row[1:])
            else:
                time = f"{row[0]} Q1-Q4"
                last_year = row[0]
                data_rows.append([time] + row[1:])
        else:
            print("x")
        break                             
    # Create DataFrame
    columns = ["Time"] + countries_with_numbers
    df = pd.DataFrame(data_rows, columns=columns)
    return df

# iter through pdf's
for key, val in tab_dict.items():
    # iter through first and second table half
    df0 = str_data_to_df(tab_dict, key, "balance", 0)
    df1 = str_data_to_df(tab_dict, key, "balance", 1)
    balance_df = pd.merge(df0, df1, on="Time", how="outer")
    
    df2 = str_data_to_df(tab_dict, key, "debt", 0)
    df3 = str_data_to_df(tab_dict, key, "debt", 1)
    debt_df = pd.merge(df2, df3, on="Time", how="outer")

11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11


# TODO: cases issue, Nan issues, time col issue

In [17]:
debt_df

,Time,Belgien (1),Deutschland (2),Estland (3),Irland (4),Griechenland (5),Spanien (6),Frankreich (7),Kroatien (8),Italien (9),...,Lettland (11),Litauen (12),Luxemburg (13),Malta (14),Niederlande (15),Österreich (16),Portugal (17),Slowenien (18),Slowakei (19),Finnland (202019)
0,2020 Q1-Q4,"111,8","68,8","18,6","58,1","207,0","120,3","114,6","86,8","154,9",...,"42,2","46,2","24,6","52,2","54,7","83,0","134,9","79,6","58,9","74,7"


In [20]:
df = pd.read_csv(data1, sep " ")
df

SyntaxError: invalid syntax (943122783.py, line 1)